# Kissing cylinders

In [ ]:
#@title Data and verification

import itertools
import logging
import time
import numpy as np
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import collections
import copy
import math
import numba

njit = numba.njit

cylinder_config_c_7 = np.array([[[-0.895079798222949, -0.218667037637932, -2.698674434206914], [ 0.455691112272712, -0.886598452747669, -0.079301915368401]], [[ 2.312489793725001, 3.147856283569415, 0.748107205326668], [-0.664105297845157, 0.595346549216158, -0.452246215805514]], [[ 3.170115696452898, -1.572941214905496, 0.275241501569553], [ 0.101170430028976, 0.365792590480751, 0.925181238913292]], [[ 1.406156005670123, -0.503509206206139, -1.138840951658002], [ 0.278750862076112, 0.957110225796107, -0.078980836715941]], [[ 3.310296433116253, -4.04403876669603 , 1.793851653115256], [ 0.391295737183334, 0.621396411266488, 0.678788734531799]], [[ 0.571615889126358, 0.70496184855727 , 0.536647055898052], [ 0.303154750428934, -0.720544668653402, 0.623628557530466]], [[ 0.91063644953714 , 1.170580327427548, -3.796585078946353], [ 0.071052005884345, 0.948268376505215, 0.30941670701495 ]]])


@njit
def get_closest_approach_midpoint_numba(p1: np.ndarray, v1: np.ndarray,
                                      p2: np.ndarray, v2: np.ndarray,
                                      epsilon_parallel: float = 1e-7) -> np.ndarray:
    """
    Calculates the midpoint of the segment of closest approach between two lines L1 and L2.
    L1(t) = p1 + t*v1
    L2(s) = p2 + s*v2
    Assumes v1, v2 are unit vectors.
    p1, p2 are points on the lines, standardized (p_i . v_i = 0).
    """
    dp = p1 - p2 # Vector from p2 to p1
    v1_dot_v2 = np.dot(v1, v2)

    if np.abs(np.abs(v1_dot_v2) - 1.0) < epsilon_parallel:
        # Parallel lines:
        # The line of midpoints between L1(t)=p1+t*v1 and L2(t)=p2+t*v1 is (p1+p2)/2 + t*v1.
        # We take the point on this line closest to the origin.
        mid_of_standardized_points = (p1 + p2) / 2.0
        m_closest_to_origin = mid_of_standardized_points - np.dot(mid_of_standardized_points, v1) * v1
        return m_closest_to_origin

    # Skew lines:
    # Solve for t_c, s_c such that C1=p1+t_c*v1 and C2=p2+s_c*v2 are closest points.
    # dp_dot_v1 = (p1-p2).v1
    # dp_dot_v2 = (p1-p2).v2
    dp_dot_v1 = np.dot(dp, v1)
    dp_dot_v2 = np.dot(dp, v2)

    denominator = 1.0 - v1_dot_v2 * v1_dot_v2
    # This denominator should not be zero if not parallel (already checked).

    # t_c is parameter for L1: p1 + t_c*v1
    # s_c is parameter for L2: p2 + s_c*v2
    # Using formulas from common sources (e.g. Eberly, Geometric Tools for Computer Graphics)
    # for parameters t1, t2 for lines P1+t1*D1 and P2+t2*D2
    # t1 = (dot(D1,D2)*dot(P2-P1,D2) - dot(P2-P1,D1)) / (1 - dot(D1,D2)^2)
    # t2 = (dot(P2-P1,D2) - dot(D1,D2)*dot(P2-P1,D1)) / (1 - dot(D1,D2)^2)
    # My v1=D1, v2=D2. My p1=P1, p2=P2. My dp = p1-p2. So (P2-P1) = -dp.
    # (P2-P1).D2 = -dp_dot_v2. (P2-P1).D1 = -dp_dot_v1.

    t_c = (v1_dot_v2 * (-dp_dot_v2) - (-dp_dot_v1)) / denominator
    s_c = ((-dp_dot_v2) - v1_dot_v2 * (-dp_dot_v1)) / denominator

    c1 = p1 + t_c * v1
    c2 = p2 + s_c * v2

    return (c1 + c2) / 2.0



@njit
def _normalize_vector_numba(v: np.ndarray) -> np.ndarray:
    """Normalizes a vector. Returns a zero vector if input is zero vector."""
    norm = np.linalg.norm(v)
    if norm < 1e-9: # Threshold for zero vector
        # Return a random unit vector as a fallback, or handle as error
        # For simplicity here, returning a fixed axis if this happens during mutation.
        # During init, we'd regenerate.
        fallback_v = np.zeros_like(v)
        if fallback_v.shape[0] > 0:
            fallback_v[0] = 1.0
        return fallback_v
    return v / norm

@njit
def _random_unit_vector_numba(dim: int) -> np.ndarray:
    """Generates a random unit vector in 'dim' dimensions."""
    v = np.empty(dim, dtype=np.float64)
    for i in range(dim):
        v[i] = np.random.normal(0,1) # Gaussian distribution
    return _normalize_vector_numba(v)

@njit
def _standardize_point_on_axis_numba(p: np.ndarray, v: np.ndarray) -> np.ndarray:
    """Given a point p and a unit direction vector v, returns the point on the line (p,v)
    that is closest to the origin. This point p_std satisfies p_std · v = 0."""
    # v must be a unit vector for this formula
    return p - np.dot(p, v) * v

@njit
def distance_between_skew_lines_numba(p1: np.ndarray, v1: np.ndarray, p2: np.ndarray, v2: np.ndarray) -> float:
    """
    Calculates the shortest distance between two skew lines.
    Line 1: L1(s) = p1 + s * v1
    Line 2: L2(t) = p2 + t * v2
    v1 and v2 must be unit vectors.
    """
    # Vector connecting the two points p1 and p2
    p1_p2 = p2 - p1

    # Cross product of direction vectors
    v1_cross_v2 = np.cross(v1, v2)
    norm_v1_cross_v2 = np.linalg.norm(v1_cross_v2)

    if norm_v1_cross_v2 < 1e-9:  # Lines are parallel
        # Distance from p2 to line (p1, v1)
        # dist = || (p2 - p1) x v1 || / ||v1||
        # Since v1 is unit vector, dist = || (p2 - p1) x v1 ||
        return np.linalg.norm(np.cross(p1_p2, v1))
    else:
        # Lines are skew or intersecting
        # Distance = | (p2 - p1) · (v1 x v2) | / ||v1 x v2||
        return np.abs(np.dot(p1_p2, v1_cross_v2)) / norm_v1_cross_v2

@njit
def calculate_arrangement_score_numba(cylinder_params: List[Tuple[np.ndarray, np.ndarray]],
                                      target_dist: float,
                                      contact_box_limit: float = 100.0) -> float: # New parameter
    """
    Calculates the score for a given list of cylinder arrangements.
    Score = -sum_{i<j} (distance_between_axes_ij - target_dist)^2
    Also heavily penalizes if contact midpoints are outside [-box_limit, box_limit]^3.
    """
    num_cylinders = len(cylinder_params)
    if num_cylinders == 0:
        # Conventionally, score for empty set can be 0 or -np.inf.
        # If AlphaEvolve might try num_cylinders=0, 0 is safer to not immediately kill exploration.
        # But for a fixed num_cylinders problem, this branch is less critical.
        return -np.inf

    total_penalty = 0.0

    # Basic validation of individual cylinder parameters
    for i in range(num_cylinders):
        p_i, v_i = cylinder_params[i]
        if not np.all(np.isfinite(p_i)) or not np.all(np.isfinite(v_i)):
            return -np.inf # Invalid coordinates
        if np.abs(np.linalg.norm(v_i) - 1.0) > 1e-6: # v_i must be unit vector
            return -np.inf
        # p_i must be standardized (p_i . v_i = 0). This is done by evolve_me.
        # Adding a check here can catch issues if standardization is missed.
        if np.abs(np.dot(p_i, v_i)) > 1e-5: # Allow slightly larger tolerance due to float math
             return -np.inf # Standardization issue


    for i in range(num_cylinders):
        for j in range(i + 1, num_cylinders):
            p1, v1 = cylinder_params[i]
            p2, v2 = cylinder_params[j]

            # Standard distance penalty
            dist_axes = distance_between_skew_lines_numba(p1, v1, p2, v2)
            penalty_ij = (dist_axes - target_dist)**2
            total_penalty += penalty_ij

            # New: Check contact point location constraint
            midpoint_of_closest_approach = get_closest_approach_midpoint_numba(p1, v1, p2, v2)

            # Check if any coordinate of the midpoint exceeds the box limit
            for coord_idx in range(midpoint_of_closest_approach.shape[0]): # Iterate 0,1,2 for 3D
                if np.abs(midpoint_of_closest_approach[coord_idx]) > contact_box_limit:
                    # This configuration is invalid due to contact point location
                    return -np.inf # Immediate rejection

    return -total_penalty



def calculate_arrangement_score_hidden(
    cylinder_params: List[Tuple[np.ndarray, np.ndarray]],
    target_dist: float = 2.0,
    contact_box_limit: float = 100.0  # Add new parameter with default
    ) -> float:
    """
    Non-Numba wrapper for validation or if Numba is unavailable.
    Ensures type compatibility for Numba JIT call.
    """
    typed_cylinder_params = numba.typed.List()

    for p_val, v_val in cylinder_params:
        # Ensure p and v are numpy arrays and of correct dimension/type
        # More robust type checking
        if not (isinstance(p_val, np.ndarray) and p_val.ndim == 1 and p_val.shape[0] == 3 and p_val.dtype == np.float64 and
                isinstance(v_val, np.ndarray) and v_val.ndim == 1 and v_val.shape[0] == 3 and v_val.dtype == np.float64):
             return -np.inf # Invalid input format for Numba
        if not np.all(np.isfinite(p_val)) or not np.all(np.isfinite(v_val)):
            return -np.inf # Contains non-finite numbers

        typed_cylinder_params.append((p_val, v_val))

    # Call Numba version with all parameters
    return calculate_arrangement_score_numba(typed_cylinder_params, target_dist, contact_box_limit)

calculate_arrangement_score_hidden(cylinder_config_c_7)

In [ ]:
#@title Visualization

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # For 3D plotting
import numpy as np
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict

# Helper function to get orthogonal vectors for cylinder mesh generation
def _get_orthogonal_vectors(v_axis: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """
    Generates two unit vectors orthogonal to v_axis and to each other.
    v_axis must be a non-zero vector.
    """
    v_axis = v_axis / np.linalg.norm(v_axis)  # Ensure unit vector

    # Create a non-parallel vector
    arbitrary_vec = np.array([1.0, 0.0, 0.0])
    if np.linalg.norm(np.cross(v_axis, arbitrary_vec)) < 1e-5:  # v_axis is parallel to arbitrary_vec
        arbitrary_vec = np.array([0.0, 1.0, 0.0]) # Use another arbitrary vector

    u_perp_raw = np.cross(v_axis, arbitrary_vec)
    u_perp = u_perp_raw / np.linalg.norm(u_perp_raw)

    w_perp = np.cross(v_axis, u_perp)
    # v_axis, u_perp are unit and orthogonal, so w_perp is unit and orthogonal to both.
    return u_perp, w_perp

def plot_cylinders(
    cylinder_params_list: List[Tuple[np.ndarray, np.ndarray]],
    radius: float = 1.0,
    cylinder_length: float = 10.0, # Length of the segment to draw for each cylinder
    num_h_points: int = 10,      # Number of points along the cylinder height
    num_theta_points: int = 20   # Number of points around the cylinder circumference
    ) -> None:
    """
    Plots a list of 3D cylinders.

    Args:
        cylinder_params_list: A list of tuples, where each tuple is (P, v).
                              P is a point on the cylinder's axis (closest to origin).
                              v is the unit direction vector of the cylinder's axis.
        radius: The radius of the cylinders.
        cylinder_length: The length of the finite cylinder segment to visualize.
        num_h_points: Resolution along the cylinder axis.
        num_theta_points: Resolution around the cylinder circumference.
    """
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')

    if not cylinder_params_list:
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.title('Cylinder Packing (0 cylinders)')
        plt.show()
        return

    # Get a colormap for distinguishing cylinders
    # cmap = plt.cm.get_cmap('viridis', len(cylinder_params_list))
    # Using matplotlib.colormaps for modern matplotlib if available
    try:
        cmap_obj = plt.colormaps['viridis'] # Access the colormap object
        # Get N distinct colors from the continuous colormap
        colors_rgba = [cmap_obj(i) for i in np.linspace(0, 0.9, len(cylinder_params_list))]
    except (AttributeError, KeyError): # Fallback for older matplotlib
        cmap_listed = plt.cm.get_cmap('viridis', len(cylinder_params_list))
        colors_rgba = [cmap_listed(i) for i in range(len(cylinder_params_list))]


    all_x_coords, all_y_coords, all_z_coords = [], [], []

    for i, (P_cyl, v_cyl) in enumerate(cylinder_params_list):
        P_cyl = np.array(P_cyl, dtype=float)
        v_cyl = np.array(v_cyl, dtype=float)

        # Ensure v_cyl is a unit vector
        norm_v_cyl = np.linalg.norm(v_cyl)
        if norm_v_cyl < 1e-9: # Avoid division by zero for zero vector
            print(f"Warning: Cylinder {i} has a zero direction vector. Skipping.")
            continue
        v_cyl = v_cyl / norm_v_cyl

        u_perp, w_perp = _get_orthogonal_vectors(v_cyl)

        # Create mesh for the cylinder segment
        # H values from -L/2 to L/2 to center the segment at P_cyl
        h_vals = np.linspace(-cylinder_length / 2.0, cylinder_length / 2.0, num_h_points)
        theta_vals = np.linspace(0, 2 * np.pi, num_theta_points)

        # H_mesh columns, Theta_mesh rows
        H_mesh, Theta_mesh = np.meshgrid(h_vals, theta_vals)

        # Parametric equations for cylinder surface:
        # X = Px + vx*h + R*(u_perpx*cos(theta) + w_perpx*sin(theta))
        # Y = Py + vy*h + R*(u_perpy*cos(theta) + w_perpy*sin(theta))
        # Z = Pz + vz*h + R*(u_perpz*cos(theta) + w_perpz*sin(theta))

        X_surf = (P_cyl[0] + v_cyl[0] * H_mesh +
                  radius * (u_perp[0] * np.cos(Theta_mesh) + w_perp[0] * np.sin(Theta_mesh)))
        Y_surf = (P_cyl[1] + v_cyl[1] * H_mesh +
                  radius * (u_perp[1] * np.cos(Theta_mesh) + w_perp[1] * np.sin(Theta_mesh)))
        Z_surf = (P_cyl[2] + v_cyl[2] * H_mesh +
                  radius * (u_perp[2] * np.cos(Theta_mesh) + w_perp[2] * np.sin(Theta_mesh)))

        ax.plot_surface(X_surf, Y_surf, Z_surf, color=colors_rgba[i], alpha=0.6,
                        rstride=1, cstride=1, linewidth=0.1, edgecolor='k', antialiased=True)

        all_x_coords.extend(X_surf.flatten())
        all_y_coords.extend(Y_surf.flatten())
        all_z_coords.extend(Z_surf.flatten())

    if not all_x_coords: # If all cylinders were skipped or list was empty
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        plt.title(f'Cylinder Packing (No valid cylinders to plot)')
        plt.show()
        return

    # Auto-scaling axes to fit all cylinders and maintain aspect ratio
    min_x, max_x = np.min(all_x_coords), np.max(all_x_coords)
    min_y, max_y = np.min(all_y_coords), np.max(all_y_coords)
    min_z, max_z = np.min(all_z_coords), np.max(all_z_coords)

    center_x, center_y, center_z = (min_x+max_x)/2, (min_y+max_y)/2, (min_z+max_z)/2

    # Calculate the half-range needed to encompass all data
    half_range = np.max([max_x-min_x, max_y-min_y, max_z-min_z]) / 2.0

    # Add a small buffer
    buffer = 0.1 * half_range
    if half_range == 0: # Handle case where all points are coincident or single cylinder of no length
        half_range = radius + cylinder_length / 2.0 # Default range based on object size
        buffer = 0.5 * half_range

    ax.set_xlim(center_x - half_range - buffer, center_x + half_range + buffer)
    ax.set_ylim(center_y - half_range - buffer, center_y + half_range + buffer)
    ax.set_zlim(center_z - half_range - buffer, center_z + half_range + buffer)

    # Setting box aspect to make display visually cubic based on the cubic limits
    current_xlim = ax.get_xlim()
    current_ylim = ax.get_ylim()
    current_zlim = ax.get_zlim()
    ax.set_box_aspect((current_xlim[1]-current_xlim[0],
                       current_ylim[1]-current_ylim[0],
                       current_zlim[1]-current_zlim[0]))


    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    plt.title(f'Cylinder Packing ({len(cylinder_params_list)} cylinders)')
    plt.show()




import matplotlib.animation as animation
from IPython.display import HTML, display # For displaying HTML5 video in Colab


def plot_cylinders_animated_colab(
    cylinder_params_list: List[Tuple[np.ndarray, np.ndarray]],
    radius: float = 1.0,
    cylinder_length: float = 10.0,
    num_h_points: int = 10,
    num_theta_points: int = 20,
    rotation_speed_degrees_per_second: float = 10.0,
    animation_duration_seconds: float = 15.0,
    fps: int = 15,
    initial_elevation: float = 30.0,
    initial_azimuth: float = -60.0
    ) -> None:
    """
    Plots a list of 3D cylinders and animates the viewpoint for Colab.
    Renders the animation as JavaScript HTML.
    """
    # Set the rcParams for animation to use jshtml for Colab
    # This is good practice, though to_jshtml() should force it.
    plt.rcParams['animation.html'] = 'jshtml' # IMPORTANT FOR COLAB

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')


    try:
        cmap_obj = plt.colormaps['viridis']
        colors_rgba = [cmap_obj(i) for i in np.linspace(0, 0.9, len(cylinder_params_list))]
    except (AttributeError, KeyError):
        cmap_listed = plt.cm.get_cmap('viridis', len(cylinder_params_list))
        colors_rgba = [cmap_listed(i) for i in range(len(cylinder_params_list))]

    all_x_coords, all_y_coords, all_z_coords = [], [], []

    for i, (P_cyl, v_cyl) in enumerate(cylinder_params_list):
        P_cyl = np.array(P_cyl, dtype=float)
        v_cyl = np.array(v_cyl, dtype=float)
        norm_v_cyl = np.linalg.norm(v_cyl)
        if norm_v_cyl < 1e-9:
            continue
        v_cyl = v_cyl / norm_v_cyl
        u_perp, w_perp = _get_orthogonal_vectors(v_cyl)

        h_vals = np.linspace(-cylinder_length / 2.0, cylinder_length / 2.0, num_h_points)
        theta_vals = np.linspace(0, 2 * np.pi, num_theta_points)
        H_mesh, Theta_mesh = np.meshgrid(h_vals, theta_vals)

        X_surf = (P_cyl[0] + v_cyl[0] * H_mesh +
                  radius * (u_perp[0] * np.cos(Theta_mesh) + w_perp[0] * np.sin(Theta_mesh)))
        Y_surf = (P_cyl[1] + v_cyl[1] * H_mesh +
                  radius * (u_perp[1] * np.cos(Theta_mesh) + w_perp[1] * np.sin(Theta_mesh)))
        Z_surf = (P_cyl[2] + v_cyl[2] * H_mesh +
                  radius * (u_perp[2] * np.cos(Theta_mesh) + w_perp[2] * np.sin(Theta_mesh)))

        surf = ax.plot_surface(X_surf, Y_surf, Z_surf, color=colors_rgba[i], alpha=0.3,
                               rstride=1, cstride=1, linewidth=0.1, edgecolor='k', antialiased=True)

        all_x_coords.extend(X_surf.flatten())
        all_y_coords.extend(Y_surf.flatten())
        all_z_coords.extend(Z_surf.flatten())

    if not all_x_coords:
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        plt.title(f'Cylinder Packing (No valid cylinders to plot)')
        plt.show()
        return

    # --- Setting Axis Limits (same as your previous function) ---
    min_x, max_x = np.min(all_x_coords), np.max(all_x_coords)
    min_y, max_y = np.min(all_y_coords), np.max(all_y_coords)
    min_z, max_z = np.min(all_z_coords), np.max(all_z_coords)
    center_x, center_y, center_z = (min_x+max_x)/2, (min_y+max_y)/2, (min_z+max_z)/2
    half_range = np.max([max_x-min_x, max_y-min_y, max_z-min_z]) / 2.0
    buffer = 0.1 * half_range
    if half_range == 0:
        half_range = radius + cylinder_length / 2.0
        buffer = 0.5 * half_range
    ax.set_xlim(center_x - half_range - buffer, center_x + half_range + buffer)
    ax.set_ylim(center_y - half_range - buffer, center_y + half_range + buffer)
    ax.set_zlim(center_z - half_range - buffer, center_z + half_range + buffer)
    current_xlim = ax.get_xlim()
    current_ylim = ax.get_ylim()
    current_zlim = ax.get_zlim()
    ax.set_box_aspect((current_xlim[1]-current_xlim[0],
                       current_ylim[1]-current_ylim[0],
                       current_zlim[1]-current_zlim[0]))
    ax.set_xlabel('X axis'); ax.set_ylabel('Y axis'); ax.set_zlabel('Z axis')
    plt.title(f'Cylinder Packing ({len(cylinder_params_list)} cylinders)')

    # --- Animation Setup ---
    total_frames = int(animation_duration_seconds * fps)
    degrees_per_frame = rotation_speed_degrees_per_second / fps

    ax.view_init(elev=initial_elevation, azim=initial_azimuth)

    def update_view(frame_num):
        current_azim = initial_azimuth + frame_num * degrees_per_frame
        ax.view_init(elev=initial_elevation, azim=current_azim)
        return fig,

    ani = animation.FuncAnimation(fig, update_view, frames=total_frames,
                                  interval=1000/fps, blit=False, repeat=False)

    # Convert the animation to JSHTML format for Colab
    print("Rendering animation for Colab (JSHTML)... This might take a moment.")
    jshtml_anim = ani.to_jshtml()
    plt.close(fig) # Close the plot to prevent static display

    # Display the JSHTML animation in Colab
    display(HTML(jshtml_anim))
    print("Animation rendering complete.")


plot_cylinders(list(cylinder_config_c_7), radius=1.0, cylinder_length=50.0)

plot_cylinders_animated_colab(cylinder_config_c_7,
                            radius=1.0,
                            cylinder_length=40.0,
                            rotation_speed_degrees_per_second=15,
                            animation_duration_seconds=24, # 360 degrees / 15 deg/sec = 24 sec for full rotation
                            fps=20)


In [ ]:
#@title Data and verification for variable radius cylinders

"""Finds the best arrangement of N mutually tangent cylinders (arbitrary radius) in 3D."""
import itertools
import logging
import time
import numpy as np
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import collections
import copy
import math
import numba

njit = numba.njit




# Constants for scoring validation (can be different from mutation/init bounds)
MIN_RADIUS_CLIP_FOR_SCORING = 0.1 # Smallest allowed radius in score function
MAX_RADIUS_CLIP_FOR_SCORING = 10.0 # Largest allowed radius in score function
CONTACT_BOX_LIMIT_CONFIG = 100.0 # Box limit for contact points




@njit
def get_closest_approach_midpoint_numba(
    p1: np.ndarray,
    v1: np.ndarray,
    p2: np.ndarray,
    v2: np.ndarray,
    epsilon_parallel: float = 1e-7,
) -> np.ndarray:
  """Calculates the midpoint of the segment of closest approach between two lines L1 and L2.

  L1(t) = p1 + t*v1 L2(s) = p2 + s*v2 Assumes v1, v2 are unit vectors. p1, p2
  are points on the lines, standardized (p_i . v_i = 0).
  """
  dp = p1 - p2  # Vector from p2 to p1
  v1_dot_v2 = np.dot(v1, v2)

  if np.abs(np.abs(v1_dot_v2) - 1.0) < epsilon_parallel:
    # Parallel lines:
    # The line of midpoints between L1(t)=p1+t*v1 and L2(t)=p2+t*v1 is (p1+p2)/2 + t*v1.
    # We take the point on this line closest to the origin.
    mid_of_standardized_points = (p1 + p2) / 2.0
    m_closest_to_origin = (
        mid_of_standardized_points - np.dot(mid_of_standardized_points, v1) * v1
    )
    return m_closest_to_origin

  # Skew lines:
  dp_dot_v1 = np.dot(dp, v1)
  dp_dot_v2 = np.dot(dp, v2)

  denominator = 1.0 - v1_dot_v2 * v1_dot_v2

  t_c = (v1_dot_v2 * (-dp_dot_v2) - (-dp_dot_v1)) / denominator
  s_c = ((-dp_dot_v2) - v1_dot_v2 * (-dp_dot_v1)) / denominator

  c1 = p1 + t_c * v1
  c2 = p2 + s_c * v2

  return (c1 + c2) / 2.0


@njit
def _normalize_vector_numba(v: np.ndarray) -> np.ndarray:
  """Normalizes a vector. Returns a zero vector if input is zero vector."""
  norm = np.linalg.norm(v)
  if norm < 1e-9:
    fallback_v = np.zeros_like(v)
    if fallback_v.shape[0] > 0:
      fallback_v[0] = 1.0
    return fallback_v
  return v / norm


@njit
def _standardize_point_on_axis_numba(
    p: np.ndarray, v: np.ndarray
) -> np.ndarray:
  """Given a point p and a unit direction vector v, returns the point on the line (p,v)

  that is closest to the origin. This point p_std satisfies p_std · v = 0.
  """
  return p - np.dot(p, v) * v


@njit
def distance_between_skew_lines_numba(
    p1: np.ndarray, v1: np.ndarray, p2: np.ndarray, v2: np.ndarray
) -> float:
  """Calculates the shortest distance between two skew lines."""
  p1_p2 = p2 - p1
  v1_cross_v2 = np.cross(v1, v2)
  norm_v1_cross_v2 = np.linalg.norm(v1_cross_v2)

  if norm_v1_cross_v2 < 1e-9:
    return np.linalg.norm(np.cross(p1_p2, v1))
  else:
    return np.abs(np.dot(p1_p2, v1_cross_v2)) / norm_v1_cross_v2


@njit
def calculate_arrangement_score_numba(
    cylinder_params: List[Tuple[np.ndarray, np.ndarray, float]], # P, V, R
    contact_box_limit: float,
    min_radius: float, # Minimum allowed radius for scoring
    max_radius: float  # Maximum allowed radius for scoring
) -> float:
  """Calculates the score for a given list of cylinder arrangements.

  Cylinders are (P, V, R).
  Score = -sum_{i<j} (distance_between_axes_ij - (r_i+r_j))^2
  Also heavily penalizes if contact midpoints are outside box or radii are invalid.
  """
  num_cylinders = len(cylinder_params)
  if num_cylinders == 0:
    return -np.inf

  total_penalty = 0.0

  for i in range(num_cylinders):
    p_i, v_i, r_i = cylinder_params[i]
    if not np.all(np.isfinite(p_i)) or not np.all(np.isfinite(v_i)) or not np.isfinite(r_i):
      return -np.inf
    if np.abs(np.linalg.norm(v_i) - 1.0) > 1e-6:
      return -np.inf
    if np.abs(np.dot(p_i, v_i)) > 1e-5:
      return -np.inf
    if not (min_radius <= r_i <= max_radius): # Radius validation
        return -np.inf

  for i in range(num_cylinders):
    for j in range(i + 1, num_cylinders):
      p1, v1, r1 = cylinder_params[i]
      p2, v2, r2 = cylinder_params[j]

      dist_axes = distance_between_skew_lines_numba(p1, v1, p2, v2)
      target_dist_ij = r1 + r2 # Dynamic target distance
      penalty_ij = (dist_axes - target_dist_ij) ** 2
      total_penalty += penalty_ij

      midpoint_of_closest_approach = get_closest_approach_midpoint_numba(
          p1, v1, p2, v2
      )

      for coord_idx in range(midpoint_of_closest_approach.shape[0]):
        if np.abs(midpoint_of_closest_approach[coord_idx]) > contact_box_limit:
          return -np.inf

  return -total_penalty


def calculate_arrangement_score_hidden(
    cylinder_params: List[Tuple[np.ndarray, np.ndarray, float]], # P, V, R
    contact_box_limit: float = 100.0,
    min_radius: float = MIN_RADIUS_CLIP_FOR_SCORING, # Use global defaults
    max_radius: float = MAX_RADIUS_CLIP_FOR_SCORING  # Use global defaults
) -> float:
  """Non-Numba wrapper for validation or if Numba is unavailable."""

  DIMENSION = 3
  num_cylinders = 9
  RADIUS_CLIP_MIN = 0.1   # Absolute minimum radius for mutation/initialization
  RADIUS_CLIP_MAX = 5.0   # Absolute maximum radius for mutation/initialization
  loaded_data = np.array(cylinder_params, dtype=object)
  current_cylinders: List[Tuple[np.ndarray, np.ndarray, float]] = []
  valid_load = False
  if isinstance(loaded_data, np.ndarray):
    if loaded_data.dtype == object: # New (P,V,R) format

      for item in loaded_data:
        if not (isinstance(item, np.ndarray) and len(item) == 3):
          current_cylinders = []
          break
        p_raw, v_raw, r_raw = item
        if not (isinstance(p_raw, list) and isinstance(v_raw, list) and isinstance(r_raw, (float, int))):
          current_cylinders = []
          break

        p = np.array(p_raw, dtype=float)
        v = np.array(v_raw, dtype=float)
        r = float(r_raw)

        if p.shape != (DIMENSION,) or v.shape != (DIMENSION,):
          current_cylinders = []; break
        if not (RADIUS_CLIP_MIN <= r <= RADIUS_CLIP_MAX): # Check against generation bounds
          print(f"Warning: Loaded radius {r} for cylinder is outside [{RADIUS_CLIP_MIN}, {RADIUS_CLIP_MAX}]. Clamping or re-evaluating.")
          # Potentially clamp or penalize, for now, let's allow if it's from a previous good solution.
          # Or, strictly: current_cylinders = []; break

        v_norm = _normalize_vector_numba(v)
        p_std = _standardize_point_on_axis_numba(p, v_norm)
        current_cylinders.append((p_std, v_norm, r))
      if len(current_cylinders) == num_cylinders : valid_load = True
  # Define Numba tuple type for (np.ndarray, np.ndarray, float)
  # Array type: C-contiguous, 1D float64 array
  cylinder_params = current_cylinders
  array_type = numba.types.Array(dtype=numba.float64, ndim=1, layout='C')
  tuple_type = numba.types.Tuple((array_type, array_type, numba.float64))
  typed_cylinder_params = numba.typed.List.empty_list(tuple_type)

  if not cylinder_params:
    return calculate_arrangement_score_numba(
        typed_cylinder_params, contact_box_limit, min_radius, max_radius
    )

  for p_val, v_val, r_val in cylinder_params:
    # Ensure p and v are C-contiguous numpy arrays of correct type and shape.
    # Radius r_val should be a float.

    if not (
        isinstance(p_val, np.ndarray)
        and p_val.ndim == 1
        and p_val.shape[0] == 3 # Assuming DIMENSION = 3
        # and p_val.dtype == np.float64 # np.ascontiguousarray handles dtype
        and isinstance(v_val, np.ndarray)
        and v_val.ndim == 1
        and v_val.shape[0] == 3 # Assuming DIMENSION = 3
        # and v_val.dtype == np.float64 # np.ascontiguousarray handles dtype
        and isinstance(r_val, (float, int, np.floating, np.integer)) # r_val is scalar
    ):
      return -np.inf  # Invalid input format

    p_val_contig = np.ascontiguousarray(p_val, dtype=np.float64)
    v_val_contig = np.ascontiguousarray(v_val, dtype=np.float64)
    r_val_float = float(r_val)



    if not (np.all(np.isfinite(p_val_contig)) and
            np.all(np.isfinite(v_val_contig)) and
            np.isfinite(r_val_float)):
      return -np.inf

    typed_cylinder_params.append((p_val_contig, v_val_contig, r_val_float))

  return calculate_arrangement_score_numba(
      typed_cylinder_params, contact_box_limit, min_radius, max_radius
  )


cylinder_config_c_9 = [([0.19857821545445875, -0.23831984674253506, -0.24522717314179707], [0.6511120386651232, -0.2084217497676763, 0.7298037320602897], 0.1525803323841557), ([-0.045817225700147575, -0.6222633532305163, -0.4434641824479314], [0.19047945804717148, -0.5789996892270514, 0.7927653725643129], 0.10952216244488278), ([0.04511053991130861, 0.002258745581368144, 0.1327198625952239], [0.3150895961875385, 0.9410429983263167, -0.12311223202992576], 0.28130367386886235), ([-0.04184799590104818, -0.4929695222129185, -0.21439853571132267], [-0.9509235907009279, 0.18770702863812652, -0.2459886095905652], 0.1298601091306748), ([-0.35528274690077316, -0.028970026215526707, 0.008740679923735735], [-0.01079665387167614, 0.40786673831291614, 0.9129776317321024], 0.10519100783876235), ([-0.4619833104980376, -0.8801089686879998, -0.5580356575434811], [0.6029986301609905, -0.6282442325369595, 0.4916318096991041], 0.32241514004377336), ([-0.9849026328771985, 0.8655202022895421, -0.2956387569232707], [0.5457084539243082, 0.3629981695308365, -0.7552712176647558], 0.982076826179542), ([0.2034187738303467, -0.5901674339738545, 0.7902692635587766], [0.5837162724755628, -0.57134244456614, -0.5769255795026095], 0.38776071759683944), ([1.0601921587661116, -0.13292772033315334, -0.9322465519973098], [-0.3970124429449542, -0.8566866695601167, -0.329346429075639], 0.9346691702719084)]

calculate_arrangement_score_hidden(cylinder_config_c_9)

In [ ]:
#@title Visualization

from typing import List, Tuple, Union
def plot_cylinders_animated_colab_varied_radii(
    cylinder_data_list: List[Tuple[Union[np.ndarray, List], Union[np.ndarray, List], float]], # P, v, radius
    default_cylinder_length: float = 10.0, # Use if individual lengths not provided
    num_h_points: int = 10,
    num_theta_points: int = 20,
    rotation_speed_degrees_per_second: float = 10.0,
    animation_duration_seconds: float = 15.0,
    fps: int = 15,
    initial_elevation: float = 30.0,
    initial_azimuth: float = -60.0,
    alpha_value: float = 0.7 # Default transparency
    ) -> None:
    """
    Plots a list of 3D cylinders with individual radii and animates for Colab.
    Input: List of tuples, each (P_coords, v_coords, radius_val).
    P_coords and v_coords can be lists or np.arrays.
    """
    plt.rcParams['animation.html'] = 'jshtml'
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    if not cylinder_data_list:
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        plt.title('Cylinder Packing (0 cylinders)')
        plt.show()
        return

    try:
        cmap_obj = plt.colormaps['viridis']
        colors_rgba = [cmap_obj(i) for i in np.linspace(0, 0.9, len(cylinder_data_list))]
    except (AttributeError, KeyError):
        cmap_listed = plt.cm.get_cmap('viridis', len(cylinder_data_list))
        colors_rgba = [cmap_listed(i) for i in range(len(cylinder_data_list))]

    all_x_coords, all_y_coords, all_z_coords = [], [], []
    max_radius_for_bounds = 0.0

    for i, data_item in enumerate(cylinder_data_list):
        if len(data_item) == 3:
            P_raw, v_raw, current_radius = data_item
            # cylinder_length could also be per-cylinder if needed, for now use default
            current_cylinder_length = default_cylinder_length
        elif len(data_item) == 4: # If you ever want to pass (P, v, radius, length)
            P_raw, v_raw, current_radius, current_cylinder_length = data_item
        else:
            print(f"Warning: Item {i} has unexpected format. Skipping.")
            continue

        P_cyl = np.array(P_raw, dtype=float)
        v_cyl = np.array(v_raw, dtype=float)

        if P_cyl.shape != (3,) or v_cyl.shape != (3,):
            print(f"Warning: P or V for cylinder {i} not 3D. Skipping.")
            continue

        norm_v_cyl = np.linalg.norm(v_cyl)
        if norm_v_cyl < 1e-9:
            print(f"Warning: Cylinder {i} has zero direction vector. Skipping.")
            continue
        v_cyl = v_cyl / norm_v_cyl

        u_perp, w_perp = _get_orthogonal_vectors(v_cyl)

        h_vals = np.linspace(-current_cylinder_length / 2.0, current_cylinder_length / 2.0, num_h_points)
        theta_vals = np.linspace(0, 2 * np.pi, num_theta_points)
        H_mesh, Theta_mesh = np.meshgrid(h_vals, theta_vals)

        # Use current_radius for this specific cylinder
        X_surf = (P_cyl[0] + v_cyl[0] * H_mesh +
                  current_radius * (u_perp[0] * np.cos(Theta_mesh) + w_perp[0] * np.sin(Theta_mesh)))
        Y_surf = (P_cyl[1] + v_cyl[1] * H_mesh +
                  current_radius * (u_perp[1] * np.cos(Theta_mesh) + w_perp[1] * np.sin(Theta_mesh)))
        Z_surf = (P_cyl[2] + v_cyl[2] * H_mesh +
                  current_radius * (u_perp[2] * np.cos(Theta_mesh) + w_perp[2] * np.sin(Theta_mesh)))

        ax.plot_surface(X_surf, Y_surf, Z_surf, color=colors_rgba[i], alpha=alpha_value,
                        rstride=1, cstride=1, linewidth=0.1, edgecolor='k', antialiased=True)

        all_x_coords.extend(X_surf.flatten())
        all_y_coords.extend(Y_surf.flatten())
        all_z_coords.extend(Z_surf.flatten())
        if current_radius > max_radius_for_bounds:
            max_radius_for_bounds = current_radius


    if not all_x_coords: # If all cylinders were skipped
        ax.set_xlabel('X'); ax.set_ylabel('Y'); ax.set_zlabel('Z')
        plt.title(f'Cylinder Packing (No valid cylinders to plot)')
        plt.show()
        return

    min_x, max_x = np.min(all_x_coords), np.max(all_x_coords)
    min_y, max_y = np.min(all_y_coords), np.max(all_y_coords)
    min_z, max_z = np.min(all_z_coords), np.max(all_z_coords)
    center_x, center_y, center_z = (min_x+max_x)/2, (min_y+max_y)/2, (min_z+max_z)/2

    half_range = np.max([max_x-min_x, max_y-min_y, max_z-min_z]) / 2.0
    buffer = 0.1 * half_range
    if half_range == 0: # Handle case of single point or zero-extent object
        # Use max_radius and default_cylinder_length for a sensible default view
        half_range = max_radius_for_bounds + default_cylinder_length / 2.0
        if half_range == 0: half_range = 5.0 # Absolute fallback
        buffer = 0.5 * half_range

    ax.set_xlim(center_x - half_range - buffer, center_x + half_range + buffer)
    ax.set_ylim(center_y - half_range - buffer, center_y + half_range + buffer)
    ax.set_zlim(center_z - half_range - buffer, center_z + half_range + buffer)

    current_xlim = ax.get_xlim(); current_ylim = ax.get_ylim(); current_zlim = ax.get_zlim()
    ax.set_box_aspect((current_xlim[1]-current_xlim[0],
                       current_ylim[1]-current_ylim[0],
                       current_zlim[1]-current_zlim[0]))
    ax.set_xlabel('X axis'); ax.set_ylabel('Y axis'); ax.set_zlabel('Z axis')
    plt.title(f'Cylinder Packing ({len(cylinder_data_list)} varied radii cylinders)')

    total_frames = int(animation_duration_seconds * fps)
    degrees_per_frame = rotation_speed_degrees_per_second / fps
    ax.view_init(elev=initial_elevation, azim=initial_azimuth)

    def update_view(frame_num):
        current_azim = initial_azimuth + frame_num * degrees_per_frame
        ax.view_init(elev=initial_elevation, azim=current_azim)
        return fig,

    ani = animation.FuncAnimation(fig, update_view, frames=total_frames,
                                  interval=1000/fps, blit=False, repeat=False)

    print("Rendering animation for Colab (JSHTML, varied radii)... This might take a moment.")
    jshtml_anim = ani.to_jshtml()
    plt.close(fig)
    display(HTML(jshtml_anim))
    print("Animation rendering complete.")


plot_cylinders_animated_colab_varied_radii(
     cylinder_config_c_9,
     default_cylinder_length=8.0, # Adjust as needed
     alpha_value=0.3, # Can be 1.0 for opaque
     animation_duration_seconds=10,
     fps=10
 )

**Prompt used** (for the unit cylinder case)

Act as an expert in optimization and 3D geometry.
Your goal is to find the best possible arrangement for num_cylinders mutually tangent unit-radius cylinders in 3D space.
You need to produce the Python code for a search function. This function must:
Search for an optimal list of num_cylinders cylinder configurations.
Each cylinder is defined by a pair (P, v):
P: a 3D np.array representing the point on the cylinder's axis closest to the origin.
v: a 3D np.array representing the unit direction vector of the cylinder's axis.
Aim to maximize a score. The score is S = -sum_ij (d_ij - 2.0)^2 for all unique pairs of cylinders (i,j), where d_ij is the shortest distance between the axes of cylinder i and cylinder j. A perfect score is 0 (all cylinders perfectly tangent).
Use the provided calculate_arrangement_score_numba(arrangement, target_dist=2.0) function to evaluate the score of any given arrangement. You do not need to implement this scoring function.
Return the best list of num_cylinders (P,v) tuples it finds. The function will have a 1000-second time limit to run. Failure to return within this time will result in its output being disregarded (equivalent to a very poor score).
Good luck!

In [ ]:
#@title Initial program used (written by Gemini)

import itertools
import logging
import time
import numpy as np
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
import collections
import copy
import math
import numba

njit = numba.njit

def evolve_me(num_cylinders: int) -> List[Tuple[np.ndarray, np.ndarray]]:
  """Searches for the best arrangement of cylinders.

  Cylinder parameters are (P, v) where P is a point on the axis (closest to
  origin) and v is the unit direction vector of the axis.
  """
  initial_search_radius: float = 5.0
  mutation_scale_p: float = 0.1
  mutation_scale_v: float = 0.1
  runtime_seconds: int = 100
  DIMENSION = 3  # Fixed for 3D space
  TARGET_AXIS_DISTANCE = 2.0  # For unit radius cylinders to be tangent

  # Try to load previous best configuration
  variable_name = f'cylinder_config_c_{num_cylinders}'

  current_cylinders: List[Tuple[np.ndarray, np.ndarray]] = []

  # This global access pattern is typical for AlphaEvolve to reuse prior results.
  if np.random.rand() < 0.99 and variable_name in globals():
    try:
      loaded_config = list(globals()[variable_name])
      if isinstance(loaded_config, list):
        for p_raw, v_raw in loaded_config:
          p = np.array(p_raw, dtype=float)
          v = np.array(v_raw, dtype=float)

          if p.shape != (DIMENSION,) or v.shape != (DIMENSION,):
            # Data corruption or wrong format, re-initialize
            current_cylinders = []
            break

          v = _normalize_vector_numba(v)  # Ensure v is unit
          p = _standardize_point_on_axis_numba(p, v)  # Ensure P is standardized
          current_cylinders.append((p, v))

        if not current_cylinders:  # If break happened
          print(f'Failed to load or parse {variable_name}, re-initializing.')
      else:  # Not a list, re-initialize
        print(f'Loaded {variable_name} is not a list, re-initializing.')
        current_cylinders = []

    except Exception as e:  # Broad catch for any parsing/conversion issues
      print(f'Error loading {variable_name}: {e}. Re-initializing.')
      current_cylinders = []

  # Initialize or complete cylinder list if needed
  if len(current_cylinders) < num_cylinders:
    num_to_add = num_cylinders - len(current_cylinders)
    for _ in range(num_to_add):
      p_init = (np.random.rand(DIMENSION) - 0.5) * 2 * initial_search_radius
      v_init = _random_unit_vector_numba(DIMENSION)
      p_init = _standardize_point_on_axis_numba(p_init, v_init)
      current_cylinders.append((p_init, v_init))
  elif len(current_cylinders) > num_cylinders:
    current_cylinders = current_cylinders[:num_cylinders]

  # Use Numba typed list for performance in score calculation
  # Note: This conversion happens once. If current_cylinders is modified,
  # it might need to be reconverted or ensure modifications maintain type.
  # Deepcopy ensures that internal numpy arrays are also copied.
  # However, Numba typed lists handle Python lists of NumPy arrays directly.

  # The `calculate_arrangement_score_numba` expects a list of tuples of numpy arrays.
  # `current_cylinders` is already in this format. Numba will JIT compile based on this.

  best_cylinders = copy.deepcopy(current_cylinders)
  # Use Numba compiled function for scoring
  best_score = calculate_arrangement_score_numba(
      numba.typed.List(best_cylinders), TARGET_AXIS_DISTANCE
  )

  current_score = best_score
  print(f'Initial score: {best_score:.4f}')

  start_time = time.time()
  eval_count = 0
  max_evals_or_time = runtime_seconds  # Use time for termination

  while time.time() - start_time < max_evals_or_time:
    candidate_cylinders = copy.deepcopy(current_cylinders)  # Work on a copy

    idx_to_mutate = np.random.randint(0, num_cylinders)
    p_old, v_old = candidate_cylinders[idx_to_mutate]

    # Mutate either P or v
    if np.random.rand() < 0.5:  # Mutate P
      p_perturbation = (np.random.rand(DIMENSION) - 0.5) * 2 * mutation_scale_p
      p_new = p_old + p_perturbation
      v_new = v_old
    else:  # Mutate v
      v_perturbation = (np.random.rand(DIMENSION) - 0.5) * 2 * mutation_scale_v
      v_new = v_old + v_perturbation
      v_new = _normalize_vector_numba(v_new)  # Ensure still unit vector
      p_new = p_old

    # Standardize P according to the (potentially new) v
    p_new = _standardize_point_on_axis_numba(p_new, v_new)
    candidate_cylinders[idx_to_mutate] = (p_new, v_new)

    # Score the candidate
    score = calculate_arrangement_score_numba(
        numba.typed.List(candidate_cylinders), TARGET_AXIS_DISTANCE
    )
    eval_count += 1

    # Simple hill climbing / simulated annealing acceptance
    if score > current_score:  # Found a better or equally good configuration
      current_cylinders = copy.deepcopy(candidate_cylinders)
      current_score = score
      if score > best_score:
        best_score = score
        best_cylinders = copy.deepcopy(candidate_cylinders)
        print(f'New best score: {best_score:.4f} (evals: {eval_count})')
    elif (
        np.random.rand() < 0.01
    ):  # Small chance to jump even if worse (helps escape local optima)
      # This is a very basic SA-like step. Could be more sophisticated.
      current_cylinders = copy.deepcopy(candidate_cylinders)
      current_score = score

    # Periodically revert to best to focus search if stuck
    if (
        np.random.rand() < 0.1 and eval_count % 100 == 0
    ):  # Less frequent reversion
      current_cylinders = copy.deepcopy(best_cylinders)
      current_score = best_score

  print(f'Final best score: {best_score:.4f}')
  print(f'Total evaluations: {eval_count}')

  # Convert Numba typed list back to Python list of tuples of arrays if necessary
  # (best_cylinders is already in this format)
  return best_cylinders


In [ ]:
#@title Code evolved by AlphaEvolve (unit cylinder case)

# Note that we forgot to tell AlphaEvolve in the prompt what global variable
# containing the previous best constructions is called.

# Luckily since it was present in the initial program, AlphaEvolve figured it out

import itertools
import logging
import time
# from scipy import integrate # Not used here
import numpy as np
# from scipy import optimize # Not used here
import warnings
import random
import re
from collections.abc import Callable, Mapping
from typing import Any, List, Tuple, Dict
# import scipy.linalg as la # Using np.linalg
import collections
import copy
import math
import numba

njit = numba.njit


@njit
def _normalize_vector_numba(v: np.ndarray) -> np.ndarray:
    """Normalize a vector using Numba."""
    norm = np.linalg.norm(v)
    if norm < 1e-9:
        # Handle near-zero vectors. Returning a default or zero vector is safer than NaN/Inf.
        # For direction vectors, a zero vector is invalid. Return a default unit vector.
        warnings.warn("Attempted to normalize a near-zero vector. Returning [1,0,0].")
        # Check if v was all zeros before returning a default
        if np.all(np.abs(v) < 1e-9):
             return np.zeros_like(v) # If input was zero, return zero
        else:
             # Try to return a vector along a standard axis if input was non-zero but tiny
             basis_vectors = np.eye(v.shape[0])
             for basis in basis_vectors:
                 if np.linalg.norm(basis) > 1e-9: # Should always be true for basis vectors
                     return basis.astype(v.dtype)
             return np.array([1.0, 0.0, 0.0], dtype=v.dtype) # Ultimate fallback


    return v / norm

@njit
def _random_unit_vector_numba(dimension: int) -> np.ndarray:
    """Generate a random unit vector in N dimensions using Numba."""
    vec = np.random.randn(dimension)
    return _normalize_vector_numba(vec) # Use the robust normalize function

@njit
def _standardize_point_on_axis_numba(p: np.ndarray, v: np.ndarray) -> np.ndarray:
    """
    Find the point on the line P + s*v that is closest to the origin (0,0,0).
    This is the projection of the origin onto the line.
    For unit vector v, the parameter s is -dot(P,v).
    The closest point Q is P + s*v = P - dot(P,v)*v.
    This is the standardized P.
    """
    # Assuming v is already normalized (handled by _normalize_vector_numba)
    # Add a check for near-zero v just in case
    v_dot_v = np.dot(v, v)
    if v_dot_v < 1e-9:
         # If v is effectively zero, P is the only point on the "axis". Closest point to origin is P.
         warnings.warn("Attempted to standardize P with a near-zero vector v. Returning P.")
         return p # Cannot standardize if v is not a valid direction

    # Robust calculation even if v is slightly off-unit
    s = -np.dot(p, v) / v_dot_v
    return p + s * v

# Helper function to calculate closest points on two lines (cylinder axes)
@njit
def _get_closest_points_on_axes(p1: np.ndarray, v1: np.ndarray, p2: np.ndarray, v2: np.ndarray) -> Tuple[np.ndarray, np.ndarray, float]:
    """
    Calculate the closest points on two lines (defined by P, v) and the distance between them.
    Line 1: L1(s) = p1 + s * v1
    Line 2: L2(t) = p2 + t * v2
    v1, v2 are assumed to be unit vectors (or close to it after standardization).
    Returns (q1, q2, distance).
    """
    dp = p1 - p2

    # Assume v1, v2 are unit vectors for performance, but use dot product for robustness
    v1_dot_v1 = np.dot(v1, v1)
    v2_dot_v2 = np.dot(v2, v2)
    v1_dot_v2 = np.dot(v1, v2)

    dp_dot_v1 = np.dot(dp, v1)
    dp_dot_v2 = np.dot(dp, v2)

    determinant = v1_dot_v1 * v2_dot_v2 - v1_dot_v2 * v1_dot_v2

    if abs(determinant) < 1e-9:  # Axes are parallel or collinear
        # For parallel lines, the shortest distance is the norm of the component
        # of p1-p2 perpendicular to the direction vector (v1 or v2).
        # dist_vec = (p1 - p2) - dot(p1-p2, v1)*v1/dot(v1,v1)
        # Handle case where v1 is near zero
        if v1_dot_v1 < 1e-9: # v1 is near zero, line 1 is effectively point p1
            dist = np.linalg.norm(p1 - p2 - np.dot(p1-p2, v2) * v2 / v2_dot_v2 if v2_dot_v2 > 1e-9 else p1 - p2)
            q1 = p1
            q2 = p2 + np.dot(p1 - p2, v2) * v2 / v2_dot_v2 if v2_dot_v2 > 1e-9 else p2
        elif v2_dot_v2 < 1e-9: # v2 is near zero, line 2 is effectively point p2
             dist = np.linalg.norm(p2 - p1 - np.dot(p2-p1, v1) * v1 / v1_dot_v1)
             q1 = p1 + np.dot(p2 - p1, v1) * v1 / v1_dot_v1
             q2 = p2
        else: # Both v1 and v2 are valid directions (parallel)
            # dist_vec = (p1 - p2) - dot(p1-p2, v1/norm(v1))*v1/norm(v1)
            # Using v1_dot_v1, v2_dot_v2 for robustness:
            dist_vec_p1_to_line2 = dp - dp_dot_v1 * v1 / v1_dot_v1
            dist = np.linalg.norm(dist_vec_p1_to_line2)
            # For the closest points on parallel lines, multiple points exist.
            # Return a pair that yields the shortest distance vector.
            # q1 = p1, q2 = p1 - dist_vec_p1_to_line2
            q1 = p1
            q2 = p1 - dist_vec_p1_to_line2 # This point should be on line 2

        return q1, q2, dist
    else: # Axes are not parallel
        # s = (dot(v1,v2)*dot(dp,v2) - dot(v2,v2)*dot(dp,v1)) / determinant
        # t = (dot(v1,v1)*dot(dp,v2) - dot(v1,v2)*dot(dp,v1)) / determinant
        # Using our variable names:
        s_val = (v1_dot_v2 * dp_dot_v2 - v2_dot_v2 * dp_dot_v1) / determinant
        t_val = (v1_dot_v1 * dp_dot_v2 - v1_dot_v2 * dp_dot_v1) / determinant

        q1 = p1 + s_val * v1
        q2 = p2 + t_val * v2

        dist = np.linalg.norm(q1 - q2)
        return q1, q2, dist


def evolve_me(num_cylinders: int) -> List[Tuple[np.ndarray, np.ndarray]]:
  """Searches for the best arrangement of cylinders.

  Cylinder parameters are (P, v) where P is a point on the axis (closest to
  origin) and v is the unit direction vector of the axis.
  """
  initial_search_radius: float = 5.0
  # --- Optimization Parameters ---
  # Annealing parameters derived from the good prior program
  initial_search_radius: float = 5.0 # Used for initial random placement
  initial_mutation_scale_p: float = 0.5 # Start with larger scale for P mutation (lateral shift)
  initial_mutation_scale_v: float = 0.5 # Start with larger scale for V mutation (angular perturbation)
  min_mutation_scale: float = 0.01 # Ensure some minimal exploration even late in the search
  initial_prob_accept_worse: float = 0.05 # Start with reasonable chance to accept worse scores
  min_prob_accept_worse: float = 0.001 # Always have a small chance to escape

  runtime_seconds: int = 1000 # Increase runtime to 1000 seconds as per requirement

  DIMENSION = 3  # Fixed for 3D space
  TARGET_AXIS_DISTANCE = 2.0  # For unit radius cylinders to be tangent

  # Try to load previous best configuration
  # AlphaEvolve framework typically names variables with a unique ID.
  # For local testing, we might use a fixed name.
  # The variable name should reflect the problem, e.g., num_cylinders.
  # Example: cylinder_config_c8_xxxx (where xxxx is a hash/id)
  # Let's assume a generic naming convention for this example.
  # NOTE: In a real AlphaEvolve run, the framework handles variable injection.
  #       The `globals().get(variable_name)` pattern is specific to that.
  #       For this standalone script, this part is illustrative.

  # This is where AlphaEvolve injects previously found good starting points.
  # `placed_cylinders_config_guid` where guid is a unique id based on contents
  # To match the example, we could use `cylinder_config_c{num_cylinders}_...`
  # Here, I'll make up a name to show the logic.
  variable_name = f'cylinder_config_c_{num_cylinders}'  # A placeholder name

  current_cylinders: List[Tuple[np.ndarray, np.ndarray]] = []

  # This global access pattern is typical for AlphaEvolve to reuse prior results.
  # Load previous best configuration if available
  variable_name = f'cylinder_config_c_{num_cylinders}'
  current_cylinders: List[Tuple[np.ndarray, np.ndarray]] = []

  # Attempt to load a previous configuration with a high probability
  if np.random.rand() < 0.99 and variable_name in globals():
    try:
      loaded_config = globals()[variable_name]
      # Validate loaded data structure, size, and types
      if (
          isinstance(loaded_config, list)
          and len(loaded_config) == num_cylinders
          and all(
              isinstance(item, tuple)
              and len(item) == 2
              and isinstance(item[0], np.ndarray) # Expect numpy arrays
              and isinstance(item[1], np.ndarray)
              and item[0].shape == (DIMENSION,)
              and item[1].shape == (DIMENSION,)
              and item[1].ndim == 1 # Ensure vector is 1D
              for item in loaded_config
          )
      ):
          temp_cylinders = []
          for p_raw, v_raw in loaded_config:
              # Ensure data types are float explicitly
              p = np.array(p_raw, dtype=float)
              v = np.array(v_raw, dtype=float)
              # Standardize loaded data
              v = _normalize_vector_numba(v)  # Ensure v is unit
              p = _standardize_point_on_axis_numba(p, v)  # Ensure P is standardized
              temp_cylinders.append((p, v))
          current_cylinders = temp_cylinders
          print(f'Successfully loaded previous config {variable_name}.')
      else:
        print(f'Loaded {variable_name} has incorrect format, size, or types. Re-initializing.')
        current_cylinders = [] # Clear if invalid
    except Exception as e: # Catch potential errors during access or processing
      print(f'Error loading or processing {variable_name}: {e}. Re-initializing.')
      current_cylinders = []

  # If loading failed or was skipped, or if the number of cylinders is wrong, initialize from scratch
  if len(current_cylinders) != num_cylinders:
      print(f'Initializing {num_cylinders} cylinders randomly.')
      current_cylinders = [] # Ensure clear if size was wrong
      for _ in range(num_cylinders):
          # Initialize points and vectors randomly
          p_init = (np.random.rand(DIMENSION) - 0.5) * 2 * initial_search_radius
          v_init = _random_unit_vector_numba(DIMENSION)
          p_init = _standardize_point_on_axis_numba(p_init, v_init)
          current_cylinders.append((p_init, v_init))

  # current_cylinders is now guaranteed to have num_cylinders items
  # in the correct (P, v) format with standardized values.

  # Use Numba typed list for performance in score calculation.
  # The score function will convert the Python list/tuples/arrays.
  # Deepcopy ensures that internal numpy arrays are also copied when creating best_cylinders.
  # Deepcopy ensures that internal numpy arrays are also copied.
  # However, Numba typed lists handle Python lists of NumPy arrays directly.

  # The `calculate_arrangement_score_numba` expects a list of tuples of numpy arrays.
  # `current_cylinders` is already in this format. Numba will JIT compile based on this.

  best_cylinders = copy.deepcopy(current_cylinders)
  # Use Numba compiled function for scoring
  best_score = calculate_arrangement_score_numba(
      numba.typed.List(best_cylinders), TARGET_AXIS_DISTANCE
  )

  current_score = best_score
  print(f'Initial score: {best_score:.4f}')

  start_time = time.time()
  eval_count = 0
  max_runtime = runtime_seconds # Use time for termination

  while time.time() - start_time < max_runtime:
    time_elapsed = time.time() - start_time
    # Annealing factor decreases from 1.0 to 0.0 over the runtime
    # Using squared factor for a slightly slower decay initially
    annealing_factor = 1.0 - min(1.0, time_elapsed / max_runtime)
    annealing_factor_sq = annealing_factor**2

    # Calculate current mutation scales and acceptance probability with annealing
    current_mutation_scale_p = initial_mutation_scale_p * annealing_factor_sq + min_mutation_scale * (1 - annealing_factor_sq)
    current_mutation_scale_v = initial_mutation_scale_v * annealing_factor_sq + min_mutation_scale * (1 - annealing_factor_sq)
    current_prob_accept_worse = initial_prob_accept_worse * annealing_factor_sq + min_prob_accept_worse * (1 - annealing_factor_sq)

    # Create a candidate by deeply copying the current state
    candidate_cylinders = copy.deepcopy(current_cylinders)

    # Create a candidate by deeply copying the current state
    candidate_cylinders = copy.deepcopy(current_cylinders)

    # --- Targeted Tangency Mutation (TTM) ---
    # Select one cylinder 'i' to mutate
    idx_to_mutate_i = np.random.randint(0, num_cylinders)
    p_i_old, v_i_old = candidate_cylinders[idx_to_mutate_i]

    # Only perform TTM if there are at least 2 cylinders
    if num_cylinders >= 2:
        # Select a target cylinder 'j' different from 'i'
        idx_to_mutate_j = idx_to_mutate_i
        # Ensure j is different from i, handle num_cylinders=2 edge case for while loop
        if num_cylinders > 1:
            while idx_to_mutate_j == idx_to_mutate_i:
                idx_to_mutate_j = np.random.randint(0, num_cylinders)
        else: # Should not happen due to num_cylinders >= 2 check, but being safe
             idx_to_mutate_j = (idx_to_mutate_i + 1) % num_cylinders


        p_j_curr, v_j_curr = candidate_cylinders[idx_to_mutate_j]

        # Calculate distance and closest points for the selected pair (i, j)
        # Use a try-except block for _get_closest_points_on_axes in case of extreme numerical issues
        try:
            q_i, q_j, d_ij = _get_closest_points_on_axes(p_i_old, v_i_old, p_j_curr, v_j_curr)

            # Decide whether to mutate P or V for cylinder 'i'
            if np.random.rand() < 0.6:  # Mutate P (lateral shift towards/away from target Q) - Slightly higher probability for P
                # Perturbation direction is from qi to qj
                perturbation_direction = q_j - q_i
                norm_perturbation_dir = np.linalg.norm(perturbation_direction)

                # Handle cases where qi and qj are coincident (e.g., axes are the same)
                if norm_perturbation_dir < 1e-9:
                    # Use a random direction perpendicular to v_i_old
                    random_vec = _random_unit_vector_numba(DIMENSION)
                    perturbation_direction = np.cross(v_i_old, random_vec)
                    norm_perturbation_dir = np.linalg.norm(perturbation_direction)
                    # Fallback if v_i_old was parallel to random_vec or v_i_old is zero
                    if norm_perturbation_dir < 1e-9:
                         basis_vectors = np.eye(DIMENSION)
                         for basis in basis_vectors:
                              temp_perp = np.cross(v_i_old, basis)
                              if np.linalg.norm(temp_perp) > 1e-9:
                                   perturbation_direction = temp_perp
                                   norm_perturbation_dir = np.linalg.norm(perturbation_direction)
                                   break
                         if norm_perturbation_dir < 1e-9: # Still zero, fallback to arbitrary direction
                             perturbation_direction = np.array([1.0, 0.0, 0.0]) * 1e-9
                             norm_perturbation_dir = 1e-9 # Set a tiny norm

                # Calculate step magnitude based on error and scale
                # Step magnitude is proportional to absolute error |d_ij - TARGET_DIST| and current_mutation_scale_p
                error_magnitude = abs(d_ij - TARGET_AXIS_DISTANCE)
                move_magnitude = error_magnitude * current_mutation_scale_p

                # Direction: towards qj if too far (d_ij > target), away from qj if too close (d_ij < target)
                move_direction = perturbation_direction / norm_perturbation_dir # Normalized direction

                if d_ij < TARGET_AXIS_DISTANCE: # Too close
                     move_direction = -move_direction # Move away

                p_i_new = p_i_old + move_direction * move_magnitude
                v_i_new = v_i_old # v remains the same for this mutation

            else:  # Mutate v (orientation)
                # Generate a random axis of rotation candidate perpendicular to v_i_old
                random_vec = _random_unit_vector_numba(DIMENSION)
                axis_of_rotation_candidate = np.cross(v_i_old, random_vec)
                norm_axis = np.linalg.norm(axis_of_rotation_candidate)

                # Handle cases where the initial random_vec was parallel to v_i_old or v_i_old is zero
                if norm_axis < 1e-9:
                     basis_vectors = np.eye(DIMENSION)
                     for basis in basis_vectors:
                         temp_axis = np.cross(v_i_old, basis)
                         if np.linalg.norm(temp_axis) > 1e-9:
                             axis_of_rotation_candidate = temp_axis
                             norm_axis = np.linalg.norm(axis_of_rotation_candidate)
                             break
                     if norm_axis < 1e-9: # Still zero, fallback to arbitrary axis
                          axis_of_rotation_candidate = np.array([1.0, 0.0, 0.0])
                          norm_axis = 1.0 # Set norm to prevent division issues below (axis will be normalized)

                axis_of_rotation_unit = axis_of_rotation_candidate / norm_axis

                # Generate angle magnitude based on error and scale
                # Angle magnitude is proportional to absolute error |d_ij - TARGET_DIST| and current_mutation_scale_v
                error_magnitude = abs(d_ij - TARGET_AXIS_DISTANCE)
                # Scale the angle by the error and the annealed scale.
                angle_magnitude = error_magnitude * current_mutation_scale_v * 1.0 # Factor 1.0 for now

                # Randomize the sign of the angle (rotate in either direction)
                angle = angle_magnitude * (1.0 if np.random.rand() < 0.5 else -1.0)

                # Use Rodrigues' rotation formula
                k = axis_of_rotation_unit # rotation axis (unit vector)
                theta = angle # rotation angle
                # Simplified formula since dot(k, v_i_old) should be zero for a valid axis perpendicular to v_i_old
                v_i_new = v_i_old * np.cos(theta) + np.cross(k, v_i_old) * np.sin(theta)
                # Re-normalize to combat potential floating point drift
                v_i_new = _normalize_vector_numba(v_i_new)

                p_i_new = p_i_old # P remains the same initially for this mutation

            # Standardize P according to the (potentially new) v
            # This finds the point on the axis (defined by p_i_new and v_i_new) closest to the origin.
            p_i_new_standardized = _standardize_point_on_axis_numba(p_i_new, v_i_new)
            candidate_cylinders[idx_to_mutate_i] = (p_i_new_standardized, v_i_new)

        except Exception as e:
            # Fallback: If geometric calculation fails, perform a simple random perturbation
            # print(f"Error in geometric mutation for pair ({idx_to_mutate_i}, {idx_to_mutate_j}): {e}. Performing random perturbation.")
            # Use a standard random mutation as fallback
            if np.random.rand() < 0.5: # Mutate P
                 # Random perturbation perpendicular to v_i_old
                 random_vec = (np.random.rand(DIMENSION) - 0.5) * 2.0
                 perturbation_direction = random_vec - np.dot(random_vec, v_i_old) * v_i_old
                 norm_perturbation = np.linalg.norm(perturbation_direction)
                 if norm_perturbation < 1e-9:
                      basis_vectors = np.eye(DIMENSION)
                      for basis in basis_vectors:
                          temp_perp = np.cross(v_i_old, basis)
                          if np.linalg.norm(temp_perp) > 1e-9:
                              perturbation_direction = temp_perp
                              norm_perturbation = np.linalg.norm(perturbation_direction)
                              break
                      if norm_perturbation < 1e-9:
                           perturbation_direction = np.array([1.0, 0.0, 0.0]) * 1e-9
                           norm_perturbation = 1e-9

                 perturbation_vector = perturbation_direction / norm_perturbation * current_mutation_scale_p
                 p_i_new = p_i_old + perturbation_vector
                 v_i_new = v_i_old
            else: # Mutate V
                 # Random rotation around random perpendicular axis
                 random_vec = _random_unit_vector_numba(DIMENSION)
                 axis_of_rotation_candidate = np.cross(v_i_old, random_vec)
                 norm_axis = np.linalg.norm(axis_of_rotation_candidate)
                 if norm_axis < 1e-9:
                      basis_vectors = np.eye(DIMENSION)
                      for basis in basis_vectors:
                          temp_axis = np.cross(v_i_old, basis)
                          if np.linalg.norm(temp_axis) > 1e-9:
                              axis_of_rotation_candidate = temp_axis
                              norm_axis = np.linalg.norm(axis_of_rotation_candidate)
                              break
                      if norm_axis < 1e-9:
                           axis_of_rotation_candidate = np.array([1.0, 0.0, 0.0])
                           norm_axis = 1.0

                 axis_of_rotation_unit = axis_of_rotation_candidate / norm_axis
                 angle = (np.random.rand() - 0.5) * 2.0 * current_mutation_scale_v
                 k = axis_of_rotation_unit
                 theta = angle
                 v_i_new = v_i_old * np.cos(theta) + np.cross(k, v_i_old) * np.sin(theta)
                 v_i_new = _normalize_vector_numba(v_i_new)
                 p_i_new = p_i_old

            # Standardize P
            p_i_new_standardized = _standardize_point_on_axis_numba(p_i_new, v_i_new)
            candidate_cylinders[idx_to_mutate_i] = (p_i_new_standardized, v_i_new)

    else: # num_cylinders < 2, no pairs to target. No mutation relevant to tangency occurs.
        # For N=0 or N=1, score is always 0. Optimization isn't strictly needed.
        # We could add a random walk here, but passing means no change in this iteration.
        # Let's add a minimal random walk just to keep some exploration for N=1.
        if num_cylinders == 1:
            if np.random.rand() < 0.5: # Mutate P
                 perturbation_vector = (np.random.rand(DIMENSION) - 0.5) * 2.0 * min_mutation_scale # Use min scale
                 p_i_new = p_i_old + perturbation_vector
                 v_i_new = v_i_old
            else: # Mutate V
                 axis_of_rotation_unit = _random_unit_vector_numba(DIMENSION)
                 axis_of_rotation_perp = np.cross(v_i_old, axis_of_rotation_unit)
                 norm_axis = np.linalg.norm(axis_of_rotation_perp)
                 if norm_axis < 1e-9:
                      basis_vectors = np.eye(DIMENSION)
                      for basis in basis_vectors:
                          temp_axis = np.cross(v_i_old, basis)
                          if np.linalg.norm(temp_axis) > 1e-9:
                              axis_of_rotation_perp = temp_axis
                              norm_axis = np.linalg.norm(axis_of_rotation_perp)
                              break
                      if norm_axis < 1e-9:
                           axis_of_rotation_unit = np.array([1.0, 0.0, 0.0])
                           norm_axis = 1.0

                 axis_of_rotation_unit = axis_of_rotation_perp / norm_axis
                 angle = (np.random.rand() - 0.5) * 2.0 * min_mutation_scale # Use min scale
                 k = axis_of_rotation_unit
                 theta = angle
                 v_i_new = v_i_old * np.cos(theta) + np.cross(k, v_i_old) * np.sin(theta)
                 v_i_new = _normalize_vector_numba(v_i_new)
                 p_i_new = p_i_old

            # Standardize P
            p_i_new_standardized = _standardize_point_on_axis_numba(p_i_new, v_i_new)
            candidate_cylinders[idx_to_mutate_i] = (p_i_new_standardized, v_i_new)
        # If num_cylinders is 0, loop condition (time) still holds, but no cylinders exist to mutate. Pass.
        pass


    # --- End of TTM Mutation ---

    # Evaluate the score of the candidate configuration
    score = calculate_arrangement_score_numba(
        numba.typed.List(candidate_cylinders), TARGET_AXIS_DISTANCE
    )
    eval_count += 1

    # Annealed acceptance criteria:
    # Always accept if score improves.
    # Accept worse scores with a probability that decreases over time.
    if score > current_score:  # Found a better or equally good configuration
      current_cylinders = copy.deepcopy(candidate_cylinders)
      current_score = score
      if score > best_score:
        best_score = score
        best_cylinders = copy.deepcopy(candidate_cylinders)
        print(f'New best score: {best_score:.4f} (evals: {eval_count}, time: {time_elapsed:.1f}s, scales P:{current_mutation_scale_p:.3f} V:{current_mutation_scale_v:.3f}, P_accept:{current_prob_accept_worse:.3f})')
    elif np.random.rand() < current_prob_accept_worse: # Annealed chance to jump if worse
      current_cylinders = copy.deepcopy(candidate_cylinders)
      current_score = score
      # Optional: print when accepting a worse score
      # print(f'Accepted worse score: {current_score:.4f} (evals: {eval_count}, time: {time_elapsed:.1f}s)')

    # Else (score <= current_score and not accepted by SA): Reject, current_cylinders remains unchanged.

    # Removed the "Worst Pair Polish" step for simplification.
    # Relying solely on Simulated Annealing with single-cylinder mutations.


    # Periodically revert to best to focus search if stuck, less frequent over time.
    # Reversion probability also decreases slightly with annealing factor.
    reversion_prob = 0.1 * annealing_factor_sq
    if np.random.rand() < reversion_prob and eval_count % 500 == 0 and time_elapsed > max_runtime * 0.05: # Revert less often early on
      # print(f'Reverting to best score: {best_score:.4f} (evals: {eval_count}, time: {time_elapsed:.1f}s)')
      current_cylinders = copy.deepcopy(best_cylinders)
      current_score = best_score # Reset current_score to the best score


  # The loop terminates after runtime_seconds. Return the best configuration found.
  print(f'Optimization finished after {runtime_seconds} seconds.')
  print(f'Final best score: {best_score:.4f}')
  print(f'Total evaluations: {eval_count}')

  # best_cylinders is already in the desired format: List[Tuple[np.ndarray, np.ndarray]]
  return best_cylinders

